In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Загружаем данные
netflix_data_path = os.path.join(os.getcwd(), 'data', 'netflix_titles.csv')
df = pd.read_csv(netflix_data_path)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [3]:
def str_to_list(sourse_str) -> list:
    if not isinstance(sourse_str, str):
        return []

    result = []
    splited_str = str(sourse_str).strip().split(',')
    for item in splited_str:
        result.append(item.upper().strip())
    return result

assert str_to_list('www, fff') == ['WWW', 'FFF']
assert str_to_list('www,fff') == ['WWW', 'FFF']
assert str_to_list(' www, fff ') == ['WWW', 'FFF']
assert str_to_list(np.nan) == []

In [4]:
df.loc[:, 'cast'] = df.loc[:, 'cast'].apply(str_to_list)
df.loc[:, 'director'] = df.loc[:, 'director'].apply(str_to_list)
df.loc[:, 'country'] = df.loc[:, 'country'].apply(str_to_list)
df.loc[:, 'listed_in'] = df.loc[:, 'listed_in'].apply(str_to_list)
df.loc[:, 'date_added'] = pd.to_datetime(df.loc[:, 'date_added'].str.strip())
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,[KIRSTEN JOHNSON],[],[UNITED STATES],2021-09-25 00:00:00,2020,PG-13,90 min,[DOCUMENTARIES],"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,[],"[AMA QAMATA, KHOSI NGEMA, GAIL MABALANE, THABA...",[SOUTH AFRICA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, TV DRAMAS, TV MYSTERIES]","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,[JULIEN LECLERCQ],"[SAMI BOUAJILA, TRACY GOTOAS, SAMUEL JOUY, NAB...",[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[CRIME TV SHOWS, INTERNATIONAL TV SHOWS, TV AC...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,[],[],[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[DOCUSERIES, REALITY TV]","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,[],"[MAYUR MORE, JITENDRA KUMAR, RANJAN RAJ, ALAM ...",[INDIA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, ROMANTIC TV SHOWS, TV...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,[DAVID FINCHER],"[MARK RUFFALO, JAKE GYLLENHAAL, ROBERT DOWNEY ...",[UNITED STATES],2019-11-20 00:00:00,2007,R,158 min,"[CULT MOVIES, DRAMAS, THRILLERS]","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,[],[],[],2019-07-01 00:00:00,2018,TV-Y7,2 Seasons,"[KIDS' TV, KOREAN TV SHOWS, TV COMEDIES]","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,[RUBEN FLEISCHER],"[JESSE EISENBERG, WOODY HARRELSON, EMMA STONE,...",[UNITED STATES],2019-11-01 00:00:00,2009,R,88 min,"[COMEDIES, HORROR MOVIES]",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,[PETER HEWITT],"[TIM ALLEN, COURTENEY COX, CHEVY CHASE, KATE M...",[UNITED STATES],2020-01-11 00:00:00,2006,PG,88 min,"[CHILDREN & FAMILY MOVIES, COMEDIES]","Dragged from civilian life, a former superhero..."


In [5]:
def get_int_from_duration(duration: str) -> int:
    if not isinstance(duration, str):
        return np.nan
    
    return int(''.join(filter(str.isdigit, duration)))

assert get_int_from_duration('10 seasons') == 10
assert get_int_from_duration('min 10') == 10
assert get_int_from_duration(' 10 ') == 10
assert get_int_from_duration('10') == 10

In [6]:
df.loc[:, 'duration season'] = df.loc[df['type'] == 'TV Show', 'duration'].apply(get_int_from_duration)
df.loc[:, 'duration min'] = df.loc[df['type'] == 'Movie', 'duration'].apply(get_int_from_duration)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration season,duration min
0,s1,Movie,Dick Johnson Is Dead,[KIRSTEN JOHNSON],[],[UNITED STATES],2021-09-25 00:00:00,2020,PG-13,90 min,[DOCUMENTARIES],"As her father nears the end of his life, filmm...",NaN,90.0
1,s2,TV Show,Blood & Water,[],"[AMA QAMATA, KHOSI NGEMA, GAIL MABALANE, THABA...",[SOUTH AFRICA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, TV DRAMAS, TV MYSTERIES]","After crossing paths at a party, a Cape Town t...",2.0,NaN
2,s3,TV Show,Ganglands,[JULIEN LECLERCQ],"[SAMI BOUAJILA, TRACY GOTOAS, SAMUEL JOUY, NAB...",[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[CRIME TV SHOWS, INTERNATIONAL TV SHOWS, TV AC...",To protect his family from a powerful drug lor...,1.0,NaN
3,s4,TV Show,Jailbirds New Orleans,[],[],[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[DOCUSERIES, REALITY TV]","Feuds, flirtations and toilet talk go down amo...",1.0,NaN
4,s5,TV Show,Kota Factory,[],"[MAYUR MORE, JITENDRA KUMAR, RANJAN RAJ, ALAM ...",[INDIA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, ROMANTIC TV SHOWS, TV...",In a city of coaching centers known to train I...,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,[DAVID FINCHER],"[MARK RUFFALO, JAKE GYLLENHAAL, ROBERT DOWNEY ...",[UNITED STATES],2019-11-20 00:00:00,2007,R,158 min,"[CULT MOVIES, DRAMAS, THRILLERS]","A political cartoonist, a crime reporter and a...",NaN,158.0
8803,s8804,TV Show,Zombie Dumb,[],[],[],2019-07-01 00:00:00,2018,TV-Y7,2 Seasons,"[KIDS' TV, KOREAN TV SHOWS, TV COMEDIES]","While living alone in a spooky town, a young g...",2.0,NaN
8804,s8805,Movie,Zombieland,[RUBEN FLEISCHER],"[JESSE EISENBERG, WOODY HARRELSON, EMMA STONE,...",[UNITED STATES],2019-11-01 00:00:00,2009,R,88 min,"[COMEDIES, HORROR MOVIES]",Looking to survive in a world taken over by zo...,NaN,88.0
8805,s8806,Movie,Zoom,[PETER HEWITT],"[TIM ALLEN, COURTENEY COX, CHEVY CHASE, KATE M...",[UNITED STATES],2020-01-11 00:00:00,2006,PG,88 min,"[CHILDREN & FAMILY MOVIES, COMEDIES]","Dragged from civilian life, a former superhero...",NaN,88.0


In [7]:
df.loc[:, 'duration min'] = df.loc[df['rating'].isin(['74 min', '84 min', '66 min']), 'rating'].apply(get_int_from_duration)
df.loc[df['rating'].isin(['74 min', '84 min', '66 min']), 'rating'] = np.nan
df.loc[df['rating'].isna(), 'rating'] = 'unknown'

In [8]:
df['id'] = df.index + 1
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration season,duration min,id
0,s1,Movie,Dick Johnson Is Dead,[KIRSTEN JOHNSON],[],[UNITED STATES],2021-09-25 00:00:00,2020,PG-13,90 min,[DOCUMENTARIES],"As her father nears the end of his life, filmm...",NaN,NaN,1
1,s2,TV Show,Blood & Water,[],"[AMA QAMATA, KHOSI NGEMA, GAIL MABALANE, THABA...",[SOUTH AFRICA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, TV DRAMAS, TV MYSTERIES]","After crossing paths at a party, a Cape Town t...",2.0,NaN,2
2,s3,TV Show,Ganglands,[JULIEN LECLERCQ],"[SAMI BOUAJILA, TRACY GOTOAS, SAMUEL JOUY, NAB...",[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[CRIME TV SHOWS, INTERNATIONAL TV SHOWS, TV AC...",To protect his family from a powerful drug lor...,1.0,NaN,3
3,s4,TV Show,Jailbirds New Orleans,[],[],[],2021-09-24 00:00:00,2021,TV-MA,1 Season,"[DOCUSERIES, REALITY TV]","Feuds, flirtations and toilet talk go down amo...",1.0,NaN,4
4,s5,TV Show,Kota Factory,[],"[MAYUR MORE, JITENDRA KUMAR, RANJAN RAJ, ALAM ...",[INDIA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, ROMANTIC TV SHOWS, TV...",In a city of coaching centers known to train I...,2.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,[DAVID FINCHER],"[MARK RUFFALO, JAKE GYLLENHAAL, ROBERT DOWNEY ...",[UNITED STATES],2019-11-20 00:00:00,2007,R,158 min,"[CULT MOVIES, DRAMAS, THRILLERS]","A political cartoonist, a crime reporter and a...",NaN,NaN,8803
8803,s8804,TV Show,Zombie Dumb,[],[],[],2019-07-01 00:00:00,2018,TV-Y7,2 Seasons,"[KIDS' TV, KOREAN TV SHOWS, TV COMEDIES]","While living alone in a spooky town, a young g...",2.0,NaN,8804
8804,s8805,Movie,Zombieland,[RUBEN FLEISCHER],"[JESSE EISENBERG, WOODY HARRELSON, EMMA STONE,...",[UNITED STATES],2019-11-01 00:00:00,2009,R,88 min,"[COMEDIES, HORROR MOVIES]",Looking to survive in a world taken over by zo...,NaN,NaN,8805
8805,s8806,Movie,Zoom,[PETER HEWITT],"[TIM ALLEN, COURTENEY COX, CHEVY CHASE, KATE M...",[UNITED STATES],2020-01-11 00:00:00,2006,PG,88 min,"[CHILDREN & FAMILY MOVIES, COMEDIES]","Dragged from civilian life, a former superhero...",NaN,NaN,8806


In [9]:
cast = df['cast'].explode().drop_duplicates().dropna().reset_index(drop=True)
cast = pd.DataFrame(cast)
cast.loc[:, 'id'] = cast.index + 1
cast

,cast,id
0,AMA QAMATA,1
1,KHOSI NGEMA,2
2,GAIL MABALANE,3
3,THABANG MOLABA,4
4,DILLON WINDVOGEL,5
...,...,...
36422,RYAN NEWMAN,36423
36423,RAAGHAV CHANANA,36424
36424,MALKEET RAUNI,36425
36425,ANITA SHABDISH,36426


In [10]:
country = df['country'].explode().drop_duplicates().dropna().reset_index(drop=True)
country = pd.DataFrame(country)
country.loc[:, 'id'] = country.index + 1
country

,country,id
0,UNITED STATES,1
1,SOUTH AFRICA,2
2,INDIA,3
3,GHANA,4
4,BURKINA FASO,5
...,...,...
118,SUDAN,119
119,PANAMA,120
120,UGANDA,121
121,EAST GERMANY,122


In [11]:
director = df['director'].explode().drop_duplicates().dropna().reset_index(drop=True)
director = pd.DataFrame(director)
director.loc[:, 'id'] = director.index + 1
director

,director,id
0,KIRSTEN JOHNSON,1
1,JULIEN LECLERCQ,2
2,MIKE FLANAGAN,3
3,ROBERT CULLEN,4
4,JOSÉ LUIS UCHA,5
...,...,...
4984,MU CHU,4985
4985,CHANDRA PRAKASH DWIVEDI,4986
4986,MAJID AL ANSARI,4987
4987,PETER HEWITT,4988


In [12]:
listed_in = df['listed_in'].explode().drop_duplicates().dropna().reset_index(drop=True)
listed_in = pd.DataFrame(listed_in)
listed_in.loc[:, 'id'] = listed_in.index + 1
listed_in

,listed_in,id
0,DOCUMENTARIES,1
1,INTERNATIONAL TV SHOWS,2
2,TV DRAMAS,3
3,TV MYSTERIES,4
4,CRIME TV SHOWS,5
5,TV ACTION & ADVENTURE,6
6,DOCUSERIES,7
7,REALITY TV,8
8,ROMANTIC TV SHOWS,9
9,TV COMEDIES,10


In [13]:
show = df.loc[:, ['id', 'title', 'type', 'date_added', 'release_year', 'rating', 'description', 'duration season', 'duration min']]
show

,id,title,type,date_added,release_year,rating,description,duration season,duration min
0,1,Dick Johnson Is Dead,Movie,2021-09-25 00:00:00,2020,PG-13,"As her father nears the end of his life, filmm...",NaN,NaN
1,2,Blood & Water,TV Show,2021-09-24 00:00:00,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2.0,NaN
2,3,Ganglands,TV Show,2021-09-24 00:00:00,2021,TV-MA,To protect his family from a powerful drug lor...,1.0,NaN
3,4,Jailbirds New Orleans,TV Show,2021-09-24 00:00:00,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo...",1.0,NaN
4,5,Kota Factory,TV Show,2021-09-24 00:00:00,2021,TV-MA,In a city of coaching centers known to train I...,2.0,NaN
...,...,...,...,...,...,...,...,...,...
8802,8803,Zodiac,Movie,2019-11-20 00:00:00,2007,R,"A political cartoonist, a crime reporter and a...",NaN,NaN
8803,8804,Zombie Dumb,TV Show,2019-07-01 00:00:00,2018,TV-Y7,"While living alone in a spooky town, a young g...",2.0,NaN
8804,8805,Zombieland,Movie,2019-11-01 00:00:00,2009,R,Looking to survive in a world taken over by zo...,NaN,NaN
8805,8806,Zoom,Movie,2020-01-11 00:00:00,2006,PG,"Dragged from civilian life, a former superhero...",NaN,NaN


In [14]:
exploded_df = df.loc[:, ['id', 'cast']].explode(['cast'])
show_cast = pd.merge(exploded_df, cast, how='inner', on=['cast'])
show_cast = show_cast.rename(columns={'id_x': 'show_id', 'id_y': 'cast_id'}).drop(columns=['cast'])
show_cast.loc[:, 'id'] = show_cast.index + 1
show_cast

,show_id,cast_id,id
0,2,1,1
1,2,2,2
2,2,3,3
3,2,4,4
4,1515,4,5
...,...,...,...
64121,8806,36423,64122
64122,8807,36424,64123
64123,8807,36425,64124
64124,8807,36426,64125


In [15]:
exploded_df = df.loc[:, ['id', 'country']].explode(['country'])
show_country = pd.merge(exploded_df, country, how='inner', on=['country'])
show_country = show_country.rename(columns={'id_x': 'show_id', 'id_y': 'country_id'}).drop(columns=['country'])
show_country.loc[:, 'id'] = show_country.index + 1
show_country

,show_id,country_id,id
0,1,1,1
1,8,1,2
2,10,1,3
3,16,1,4
4,28,1,5
...,...,...,...
10014,8457,119,10015
10015,8478,120,10016
10016,8703,121,10017
10017,8746,122,10018


In [16]:
exploded_df = df.loc[:, ['id', 'director']].explode(['director'])
show_director = pd.merge(exploded_df, director, how='inner', on=['director'])
show_director = show_director.rename(columns={'id_x': 'show_id', 'id_y': 'director_id'}).drop(columns=['director'])
show_director.loc[:, 'id'] = show_director.index + 1
show_director

,show_id,director_id,id
0,1,1,1
1,3,2,2
2,1237,2,3
3,2669,2,4
4,6,3,5
...,...,...,...
6973,8793,4985,6974
6974,8799,4986,6975
6975,8802,4987,6976
6976,8806,4988,6977


In [17]:
exploded_df = df.loc[:, ['id', 'listed_in']].explode(['listed_in'])
show_listed_in = pd.merge(exploded_df, listed_in, how='inner', on=['listed_in'])
show_listed_in = show_listed_in.rename(columns={'id_x': 'show_id', 'id_y': 'listed_in_id'}).drop(columns=['listed_in'])
show_listed_in.loc[:, 'id'] = show_listed_in.index + 1
show_listed_in

,show_id,listed_in_id,id
0,1,1,1
1,17,1,2
2,46,1,3
3,69,1,4
4,89,1,5
...,...,...,...
19318,7722,42,19319
19319,8190,42,19320
19320,8542,42,19321
19321,8600,42,19322
